# find motifs associted with methylation

In [53]:
from core.genomic_utilities.methyl.epifrag import MethylAlignmentFile, ConsensusMethylatedFragment, MethylStatus
# https://github.com/freenome/research/blob/6c4b7c139443bb31299b895e80fba2975d9d8fe4/core/genomic_utilities/methyl/epifrag.py#L211

from core.genomic_utilities.genomic_utilities import revcomp

#load reference fasta
from configs.project_configs import GENOME_REFERENCE_FASTA as REF

from collections import defaultdict, Counter
from contextlib import closing
import pyfaidx

In [ ]:
# APOBEC enzyme
#https://www.neb.com/tools-and-resources/feature-articles/enzymatic-methyl-seq-the-next-generation-of-methylome-analysis

In [1]:
# create a test bam file, generated with 0% methyl control w/V1 panel
!gsutil cp gs://seq-data-us-1/bam/190806_A00225_0338_AHKWNHDMXX/744ab6d3-a365-59e5-a85e-043589e86e07/S_31_I_41-41_CATTGACG-CATTGACG_S31.marked.bam ./zero_control_v1_panel.bam

Copying gs://seq-data-us-1/bam/190806_A00225_0338_AHKWNHDMXX/744ab6d3-a365-59e5-a85e-043589e86e07/S_31_I_41-41_CATTGACG-CATTGACG_S31.marked.bam...
| [1 files][  2.0 GiB/  2.0 GiB]   22.6 MiB/s                                   
Operation completed over 1 objects/2.0 GiB.                                      


In [2]:
# subsample
!samtools view -h zero_control_v1_panel.bam | head -n 1000000 > test_zero_control_v1_panel.sam
!samtools view -h -b test_zero_control_v1_panel.sam > test_zero_control_v1_panel.bam

samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1


In [26]:
#read in bed file of regions of interest 

#v1 panel
regions = defaultdict(list)
with open("../methyl_panel_v1_target_twist.bed", "r") as fin:
    for line in fin:
        chrom, start, stop, name = line.split("\t")[0:4]
        regions[chrom].append((int(start), int(stop), name))
print(len(regions)) 
print(regions["chr1"][0:3])

23
[(632108, 632228, 'NM_001005277.1;OR4F16;54427'), (935143, 935263, 'NM_152486.2;SAMD11;-9403'), (943939, 944095, 'NM_015658.3;NOC2L;15205')]


In [60]:
# read in bam file as MethylAlignmentFile and process reads


def find_motif(fragment, ref, pos):
    '''
    the fragment sequence is from the reference which is always from the positive strand
    the pos of the methylated cpg is always on the positive strand
    therefore this motif finder only takes into acount motifs found for methylation on the positive strand
    '''
    seq = ref[fragment.contig][fragment.start:fragment.end].seq.upper()
    relative_pos = pos - fragment.start
    return seq[(relative_pos - 3):(relative_pos + 4)]  # seven bp motif surrounding the methylated C
    
    

def process_region(methyl_file, region_info, ref_fasta):
    
    chrom, start, end, name = region_info
    
    region_motif_dicts = {}
    
    # extend the start and end to include fragments that overlap with region of interest with only one read
    min_start = start - 300
    max_end = end + 300

    region_motif_counts = defaultdict(lambda: 0)

    for methyl_frag in methyl_file.fetch(contig=chrom, start=min_start, stop=max_end):
        # methyl_frag is ConsensusMethylatedFragment class, the ref seq between the two r1r2 read ends if reads pass filter 
        fragment = methyl_frag.support_fragments[0] # list of fragments that go into the consensus call
        #print(fragment)

        # get all motifs in the read. for each methylated CpG get the surrounding motif(defined above)
        for pos, methyl_status in fragment.get_methyl_status_loc_map().items():  # return dict of cpg sites and their methylatioin status
            if methyl_status == MethylStatus.m: 
                motif = find_motif(fragment, ref, pos)
                region_motif_counts[motif] += 1
                #print(motif)
        
    return region_motif_counts

In [62]:
test_bam = "./test_zero_control_v1_panel.bam"
test_chrm = "chr1"

with closing(pyfaidx.Fasta(REF)) as ref:
    with MethylAlignmentFile(
        alignment_file=test_bam,
        ref=ref,
        ignore_last_n_bp_fragment=0,
        skip_duplicate_removal=False,
        min_mapq=50,
        max_unconverted_chh_fraction=0.15,
        fetch_consensus=True,
        ) as methyl_file:
            
        c = 0
        region_motif_dicts = {}
        for region in regions[test_chrm]:
            if c % 100 == 0:
                print(f"regions processed: {c}")
            region_motif_counts = process_region(methyl_file, region_info=((test_chrm,) + region), ref_fasta=ref)
            region_motif_dicts[name] = region_motif_counts
            c += 1

regions processed: 0


KeyboardInterrupt: 

In [ ]:
# for each region fetch overlapping reads

In [ ]:
# for each read pair in region 
    # make into class MethylFragment (epifrag)
    # get list of methylated cpg positions ()
    # get reference fasta sequence btw start/stop of fragment
    # get motif surrounding each cpg
    # add motif count to dictionary for region: motif:count
    # output dict for region

In [ ]:
# combine dict counts for all regions
# write counts for all regions into file
#out put one counts dict per file


In [ ]:
# run on multiple 0% methyl files

In [ ]:
# make heatmap of samples vs motifs and prevalence around cpg sites